# 4 - Feras Formidáveis

## 4.2  Stop right now, thank you very much

### Introdução:

O objetivo desta atividade é implementar uma estratégia de Parada Antecipada (Early Stopping) no
processo de treino da rede neural feita em PyTorch. Utilizou-se dados simples para verificar o funcionamento, empregando X camadas com uma função de ativação linear. Boa parte do algoritmo foi adaptada do material de aula do Daniel Cassar [1].



A Parada Antecipada é um método para estimar um número de épocas adequado no treinamento do algoritmo, evitando que ele sofra subajuste ou sobre-ajuste.

### Desenvolvimento:

Importando as bibliotecas necessárias [2-5]:

In [23]:
import torch
import torch.nn as nn
import torch.optim as optim

Definindo a rede neural, baseada em uma Multi Layer Perceptron (MLP), em que aplica-se transformações lineares nos dados das camadas com base em uma função de ativação

In [4]:
class MLP(nn.Module):
    def __init__(
        self, num_dados_entrada, camadas_ocultas, funcao_ativacao, num_targets
    ):
        # o método super recupera todas as funções anteriores (no caso da classe Module)
        super().__init__()

        arquitetura = []

        # primeira camada oculta
        arquitetura.append(nn.Linear(num_dados_entrada, camadas_ocultas[0]))
        arquitetura.append(funcao_ativacao)

        # demais camadas ocultas
        for i in range(1, len(camadas_ocultas)):
            arquitetura.append(nn.Linear(camadas_ocultas[i-1], camadas_ocultas[i]))
            arquitetura.append(funcao_ativacao)

        # camada de saída
        arquitetura.append(nn.Linear(camadas_ocultas[-1], num_targets))

        self.camadas = nn.Sequential(*arquitetura)

    # similar ao método __call__    
    def forward(self, x):
        x = self.camadas(x)
        return x

Instanciando a MLP, contendo 4 dados de entrada, 2 camadas ocultas, função de ativação sigmoidal e 1 dado como output

In [5]:
NUM_DADOS_DE_ENTRADA = 4
CAMADAS_OCULTAS = [2, 3]
FUNCAO_ATIVACAO = nn.Sigmoid()
NUM_DADOS_DE_SAIDA = 1  

minha_mlp = MLP(
    NUM_DADOS_DE_ENTRADA, CAMADAS_OCULTAS, FUNCAO_ATIVACAO, NUM_DADOS_DE_SAIDA
)

Definindo a classe Camada [1], em que os neurônios são agrupados e passando a informação através de camadas ocultas:

In [7]:
# learning rate (ln)
TAXA_DE_APRENDIZADO = 0.001 

# modo de atualizar os parâmetros, usou-se o estocástico gradient descent (SGD)
otimizador = optim.SGD(minha_mlp.parameters(), lr=TAXA_DE_APRENDIZADO)

# função a ser minimizada, usou-se a do erro quadrático médio (MSE)
fn_perda = nn.MSELoss()

In [16]:
x = [
  [1.0, 0.0, -1.0, 2.3],
  [3.0, -1.0, 0.5, 9.1],
  [0.7, 8.6, 3.0, 2.6],
  [1.0, 1.0, -1.0, 4.4],
]

y_true = [1, 0, 0.2, 0.5]

x = torch.tensor(x)
y_true = torch.tensor(y_true).view(-1,1)

Por fim, definindo nossa rede neural classificadora, que vai organizar nossos dados em camadas 

In [18]:
# período de treinamento
NUM_EPOCAS = 100

# redundante, mas avisa que está na etapa de treinamento
minha_mlp.train()

for epoca in range(NUM_EPOCAS):
    # forward pass
    y_pred = minha_mlp(x)

    # zero grad
    otimizador.zero_grad()

    # loss
    loss = fn_perda(y_true, y_pred)

    # backpropagation
    loss.backward()

    # atualiza parâmetros - passo
    otimizador.step()

    # mostra resultado (opcional)
    print(epoca, loss.data)

0 tensor(0.2231)
1 tensor(0.2226)
2 tensor(0.2220)
3 tensor(0.2214)
4 tensor(0.2209)
5 tensor(0.2203)
6 tensor(0.2198)
7 tensor(0.2192)
8 tensor(0.2187)
9 tensor(0.2181)
10 tensor(0.2176)
11 tensor(0.2171)
12 tensor(0.2165)
13 tensor(0.2160)
14 tensor(0.2155)
15 tensor(0.2150)
16 tensor(0.2145)
17 tensor(0.2139)
18 tensor(0.2134)
19 tensor(0.2129)
20 tensor(0.2124)
21 tensor(0.2119)
22 tensor(0.2115)
23 tensor(0.2110)
24 tensor(0.2105)
25 tensor(0.2100)
26 tensor(0.2095)
27 tensor(0.2090)
28 tensor(0.2086)
29 tensor(0.2081)
30 tensor(0.2076)
31 tensor(0.2072)
32 tensor(0.2067)
33 tensor(0.2063)
34 tensor(0.2058)
35 tensor(0.2054)
36 tensor(0.2049)
37 tensor(0.2045)
38 tensor(0.2041)
39 tensor(0.2036)
40 tensor(0.2032)
41 tensor(0.2028)
42 tensor(0.2023)
43 tensor(0.2019)
44 tensor(0.2015)
45 tensor(0.2011)
46 tensor(0.2007)
47 tensor(0.2003)
48 tensor(0.1999)
49 tensor(0.1995)
50 tensor(0.1991)
51 tensor(0.1987)
52 tensor(0.1983)
53 tensor(0.1979)
54 tensor(0.1975)
55 tensor(0.1971)
56

In [19]:
x_teste = [
  [2.1, 4.8, -0.4, -1.0],
  [2.0, -1.9, 3.1, 7.3],
  [0.0, 1.9, 8.6, 6.4],
]

y_teste = [1.3, 0.3, 8.4]

x_teste = torch.tensor(x_teste)
y_teste = torch.tensor(y_teste).view(-1,1)

In [20]:
# evaluate, avisa que a mlp está na etapa de previsão e mostra informações
minha_mlp.eval()

MLP(
  (camadas): Sequential(
    (0): Linear(in_features=4, out_features=2, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=2, out_features=3, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=3, out_features=1, bias=True)
  )
)

Carregando o dataset próprio da biblioteca *Seaborn* relacionado a plantas [6], contendo características como comprimento e largura das pétalas e sépalas de uma flor, indicando a espécie da planta

In [21]:
# dentro do with indica que não usará os gradientes locais (no_grad()), pois já otimizou e gasta recurso
with torch.no_grad():
    y_pred = minha_mlp(x_teste)

Separando o dataset em dados de entrada e os valores reais, a fim de comparar com as previsões do modelo, além de definir uma rede neural classificadora com base na arquitetura da rede escolhida. De acordo com o dataset, há 4 atributos do problema, sendo os dados de entrada, e o algoritmo tem 1 dado de saída contendo a previsão da espécie (valor entre 0 e 1, sendo setosa mais próximo de 0 e virginica mais próximo de 1). Note que o valor real *y_true* foi atribuido na classe Valor, para permitir calcular a função de perda e os gradientes locais.

In [22]:
y_pred

tensor([[0.2127],
        [0.2488],
        [0.2660]])

Note que todos os valores estão bem próximos e em apenas uma categoria, indicando que o modelo só previu dados para o rótulo 0 (espécie setosa), mesmo com os dados de entrada contendo plantas das 2 espécies

### Conclusão:

Foi possível treinar uma rede neural em python puro para tarefas de classificação, embora ela não esteja bem otimizada, aumentando a perda ao longo do treinamento e prevendo apenas um rótulo para os dados do problema. Mesmo assim, foi interessante aprender sobre esse tipo de problema, aumentando meu conhecimento.

### Referências:

[1] CASSAR, Daniel. "ATP-303 NN 4.2 - Notebook MLP.ipynb". Material de Aula, 2025.

[2] Biblioteca Math. https://docs.python.org/3/library/math.html

[3] Biblioteca Random. https://docs.python.org/3/library/random.html

[4] Biblioteca Pandas. https://pandas.pydata.org/docs/user_guide/index.html#user-guide

[5] Biblioteca Seaborn. https://seaborn.pydata.org/

[6] Dataset estudado. https://archive.ics.uci.edu/dataset/53/iris

[7] Função de perda analisada. https://medium.com/ensina-ai/uma-explicação-visual-para-função-de-custo-binary-cross-entropy-ou-log-loss-eaee662c396c